<a href="https://colab.research.google.com/github/adasey/MachineLearn/blob/main/tensor/tensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
from __future__ import print_function # 이 구문은 나중 파이썬 버전의 (지금이 2면 3버전의) import된 구문을 사용하겠다는 뜻

import math # 수학적 계산을 위한 lib

from IPython import display # 테이블을 보기 위한 lib
from matplotlib import cm 
from matplotlib import gridspec # 격자 생성 - 함수를 그물모양의 점으로 표시할 때 사용
from matplotlib import pyplot as plt
import numpy as np # 무작위 배치 등 표에 대한 수학적 계산 필요 시
import pandas as pd # 표 생성 시 정의되는 값
from sklearn import metrics
%tensorflow_version 1.x
import tensorflow as tf
from tensorflow.python.data import Dataset

learn = 0.0000001
clip = 5.0

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

#테이블 생성
california_housing_dataframe = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",") # 구글의 데이터 베이스 사용
california_housing_dataframe = california_housing_dataframe.reindex(np.random.permutation(california_housing_dataframe.index)) # 값에 대해 불러오기 전에 먼저 정렬에 대한 무작위 배치 실행
california_housing_dataframe["median_house_value"] /= 1000.0 # 단순히 계산의 쉽기를 위해 사용
california_housing_dataframe.describe() # 테이블에서 열의 수, 평균, 표준편차, 최소, 최대, 각 분위 값 등 표현

#모델 생성 {
my_feature = california_housing_dataframe[["total_rooms"]] # 1-1. 입력에 대한 정의 : total_rooms
feature_columns = [tf.feature_column.numeric_column("total_rooms")] # 1-2. 데이터 유형의 설명 열 numeric_column을 통해 특성 열 정의 -> 데이터가 숫자.
targets = california_housing_dataframe["median_house_value"] # 2. 타겟에 대한 정의

my_optimizer = tf.train.GradientDescentOptimizer(learning_rate = learn) # SGD를 구현하기 위해 GranientDescentOptimizer를 활용해 경사 단계의 크기를 조절
my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, clip) # 위의 값의 경사에 대해 조절하는 경사 제어값
linear_regressor = tf.estimator.LinearRegressor(feature_columns = feature_columns, optimizer = my_optimizer) # 최적화를 위한 모델과 라벨과 타겟의 값을 선형회귀모델 구성.

def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None): # 이후의 실습도 이 입력합수를 지속 사용
    """Trains a linear regression model of one feature.

    Args:
      features: pandas DataFrame of features
      targets: pandas DataFrame of Targets
      batch_size: Size of batches to be passed to the model
      shuffle: True or False. Whether to shuffle the data.
      num_epochs: Number of epochs for which data should be repeated. None = repeat indefinitely
    Returns:
      Tuple of (features, labels) for next data batch
    """

    features = {ket:np.array(value) for ket,value in dict(features).items()} # pandas자료를 numpy의 배열로 변경
    ds = Dataset.from_tensor_slices((features, targets)) # dataset의 구조. 일괄 처리/반복을 설정 + 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs) 
    if shuffle:
      ds = ds.shuffle(buffer_size=10000) # 지정 시엔 데이터 섞기

    features, labels = ds.make_one_shot_iterator().get_next() # 다음 일관된 데이터로 return
    return features, labels
#}

#모델 학습
_ = linear_regressor.train(
    input_fn = lambda:my_input_fn(my_feature, targets), steps=100) # https://www.tensorflow.org/programmers_guide/datasets 참조

#모델 예측
